<a href="https://colab.research.google.com/github/negl85/DS_C2_SC2_NANCYGAMEZ/blob/main/Solucion_Reto_SC_63_Nancy_Gamez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import boxcox
from sklearn.preprocessing import LabelEncoder

# Enlace compartido al archivo CSV en Google Drive (modificado para que apunte directamente al archivo)
url_csv = "https://drive.google.com/uc?export=download&id=1TOcljFt08cx98toDGo-wIJzs44fHcvt4"

# Cargar el DataFrame desde el archivo CSV en Google Drive
data = pd.read_csv(url_csv)

# Imprimir las primeras filas del DataFrame para asegurarte de que se haya cargado correctamente
print(data.head())

# 1. Número de registros
num_registros = len(data)

# 2. Total de variables
total_variables = len(data.columns)

# 3. Tipo de cada variable
tipos_variables = data.dtypes

# 4. Cantidad de datos perdidos de cada variable en caso de que existan
datos_perdidos = data.isnull().sum()

# Imprimir la información obtenida
print("Número de registros:", num_registros)
print("Total de variables:", total_variables)
print("Tipo de cada variable:")
print(tipos_variables)
print("Cantidad de datos perdidos de cada variable en caso de que existan:")
print(datos_perdidos)

# Seleccionar las columnas que son variables categóricas
categoricas = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']

# Inicializar el codificador de etiquetas
label_encoder = LabelEncoder()

# Aplicar Label Encoder a cada columna categórica
for columna in categoricas:
    data[columna] = label_encoder.fit_transform(data[columna])

# Imprimir las primeras filas para verificar
print(data.head())

# Visualizar histogramas para todas las variables numéricas
#for col in data.select_dtypes(include=['float64', 'int64']).columns:
#    plt.figure(figsize=(8, 4))
#    sns.histplot(data[col], kde=True, color='skyblue')
#    plt.title(f'Histograma de {col}')
#    plt.show()

# Calcular la asimetría de las variables numéricas
asimetria = data.select_dtypes(include=['float64', 'int64']).skew()

# Inicializar una lista para almacenar el tipo de sesgo de cada variable
tipos_sesgo = []

# Determinar el tipo de sesgo para cada variable
for var, skew in asimetria.items():
    if skew > 0:
        tipos_sesgo.append((var, "sesgo a la derecha"))
    elif skew < 0:
        tipos_sesgo.append((var, "sesgo a la izquierda"))
    else:
        tipos_sesgo.append((var, "no tiene sesgo"))

# Imprimir el tipo de sesgo de cada variable
print("Tipo de sesgo de cada variable:")
for var, sesgo in tipos_sesgo:
    print(f"{var}: {sesgo}")

# Seleccionar las variables con sesgo a la derecha
variables_con_sesgo_derecha = ['balance', 'age', 'day', 'duration', 'campaign', 'pdays', 'previous']

# Aplicar transformación logarítmica a cada variable con sesgo a la derecha
for var in variables_con_sesgo_derecha:
    data[f'{var}_log'] = np.log(data[var] + 1)  # Sumamos 1 para evitar el logaritmo de 0

# Calcular la asimetría de las variables originales
asimetria_original = data.select_dtypes(include=['float64', 'int64']).skew()

data[f'balance_boxcox'], _ = boxcox(data['balance'] - data['balance'].min() + 1)

# Aplicar transformación Box-Cox a las variables con sesgo a la derecha
for var in variables_con_sesgo_derecha:
    data[f'{var}_boxcox'], _ = boxcox(data[var] - data[var].min() + 1)  # Sumamos 1 para asegurar que no haya ceros

# Calcular la asimetría de las variables transformadas
asimetria_transformada = data[[f'{var}_boxcox' for var in variables_con_sesgo_derecha]].skew()

# Comparar la asimetría original con la asimetría después de la transformación
for var, skew_original, skew_transformada in zip(variables_con_sesgo_derecha,
                                                  asimetria_original,
                                                  asimetria_transformada):
    print(f"Variable: {var}")
    print(f"Asimetría original: {skew_original}")
    print(f"Asimetría después de la transformación: {skew_transformada}")

# Calcular la asimetría de las variables transformadas
asimetria_transformada = data[[f'{var}_boxcox' for var in variables_con_sesgo_derecha]].skew()

# Determinar si las variables transformadas tienen sesgo
for var, skew_transformada in zip(variables_con_sesgo_derecha, asimetria_transformada):
    if abs(skew_transformada) < 0.5:
        print(f"Variable '{var}' después de la transformación no tiene sesgo (asimetría: {skew_transformada})")
    else:
        print(f"Variable '{var}' después de la transformación tiene sesgo (asimetría: {skew_transformada})")

# Separar las variables de entrada (X) y la variable de salida (y)
X = data.drop('y', axis=1)  # Variables de entrada
y = data['y']               # Variable de salida

# Imprimir las primeras filas para verificar
print(X.head())
print(y.head())

from sklearn.model_selection import train_test_split

# Dividir los datos en conjunto de entrenamiento (60%) y conjunto restante (40%)
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.4, random_state=42)

# Dividir el conjunto restante en conjunto de validación (50%) y conjunto de prueba (50%)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42)

# Imprimir las formas de los conjuntos resultantes para verificar
print("Forma del conjunto de entrenamiento:", X_train.shape, y_train.shape)
print("Forma del conjunto de validación:", X_val.shape, y_val.shape)
print("Forma del conjunto de prueba:", X_test.shape, y_test.shape)

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix

# Inicializar el modelo HistGradientBoostingRegressor
regressor_model = HistGradientBoostingRegressor()

# Aplicar el modelo al conjunto de entrenamiento
regressor_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
y_val_pred = regressor_model.predict(X_val)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_val, y_val_pred)

# Calcular el coeficiente de determinación (R²)
r2 = r2_score(y_val, y_val_pred)

# Convertir las predicciones a clases binarias (ejemplo)
y_val_pred_binary = [1 if pred > 0.5 else 0 for pred in y_val_pred]

# Calcular la precisión del modelo
accuracy = accuracy_score(y_val, y_val_pred_binary)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_val, y_val_pred_binary)

# Imprimir las métricas de evaluación del modelo
print("Error Cuadrático Medio (MSE):", mse)
print("Coeficiente de Determinación (R²):", r2)
print("Precisión del Modelo:", accuracy)
print("Matriz de Confusión:")
print(conf_matrix)

# Verificar si hay valores NaN en todas las columnas
nan_columns = data.columns[data.isna().any()].tolist()
print("Columnas con valores NaN:", nan_columns)

# Contar la cantidad de valores NaN por columna
nan_count = data.isna().sum()
print("Cantidad de valores NaN por columna:\n", nan_count)

# Eliminar la columna con muchos valores NaN
data_sin_nan_columna = data.dropna(axis=1)

# Imprimir el DataFrame resultante
print(data_sin_nan_columna.head())

# Verificar si hay valores NaN en todas las columnas
nan_columns_after_drop = data_sin_nan_columna.columns[data_sin_nan_columna.isna().any()].tolist()
print("Columnas con valores NaN después de eliminar la columna:", nan_columns_after_drop)

# Contar la cantidad de valores NaN por columna
nan_count_after_drop = data_sin_nan_columna.isna().sum()
print("Cantidad de valores NaN por columna después de eliminar la columna:\n", nan_count_after_drop)

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Definir la arquitectura de la red neuronal
model = Sequential([
    Dense(64, activation='relu', input_shape=(num_features,)),
    Dropout(0.5),  # Agregar capa de Dropout para regularización
    Dense(32, activation='relu'),
    Dropout(0.5),  # Agregar capa de Dropout para regularización
    Dense(1, activation='sigmoid')  # Capa de salida con activación sigmoide para clasificación binaria
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), verbose=True)

from sklearn.metrics import accuracy_score, confusion_matrix

# Evaluar el modelo en el conjunto de datos de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print("Precisión del modelo en el conjunto de prueba:", accuracy)

# Realizar predicciones en el conjunto de datos de prueba
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convertir las probabilidades en clases binarias

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_binary)
print("Matriz de confusión:")
print(conf_matrix)

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix

# Inicializar el modelo HistGradientBoostingRegressor
regressor_model = HistGradientBoostingRegressor()

# Aplicar el modelo al conjunto de entrenamiento
regressor_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
y_val_pred = regressor_model.predict(X_val)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_val, y_val_pred)

# Calcular el coeficiente de determinación (R²)
r2 = r2_score(y_val, y_val_pred)

# Convertir las predicciones a clases binarias (ejemplo)
y_val_pred_binary = [1 if pred > 0.5 else 0 for pred in y_val_pred]

# Calcular la precisión del modelo
accuracy = accuracy_score(y_val, y_val_pred_binary)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_val, y_val_pred_binary)

# Imprimir las métricas de evaluación del modelo en el conjunto de validación
print("Métricas de evaluación en el conjunto de validación:")
print("Error Cuadrático Medio (MSE):", mse)
print("Coeficiente de Determinación (R²):", r2)
print("Precisión del Modelo en el conjunto de validación:", accuracy)
print("Matriz de Confusión en el conjunto de validación:")
print(conf_matrix)

# Realizar predicciones en el conjunto de prueba
y_test_pred = regressor_model.predict(X_test)

# Convertir las predicciones a clases binarias (ejemplo)
y_test_pred_binary = [1 if pred > 0.5 else 0 for pred in y_test_pred]

# Calcular la precisión del modelo en el conjunto de prueba
test_accuracy = accuracy_score(y_test, y_test_pred_binary)

# Calcular la matriz de confusión en el conjunto de prueba
conf_matrix_test = confusion_matrix(y_test, y_test_pred_binary)

# Imprimir las métricas de evaluación del modelo en el conjunto de prueba
print("\nMétricas de evaluación en el conjunto de prueba:")
print("Precisión del Modelo en el conjunto de prueba:", test_accuracy)
print("Matriz de Confusión en el conjunto de prueba:")
print(conf_matrix_test)


   age            job  marital  education default  balance housing loan  \
0   31  self-employed  married   tertiary      no     2666      no   no   
1   29     unemployed   single    unknown      no     1584      no   no   
2   41    blue-collar  married  secondary      no     2152     yes   no   
3   50    blue-collar  married  secondary      no       84     yes   no   
4   40         admin.  married  secondary      no        0      no   no   

    contact  day month  duration  campaign  pdays  previous poutcome    y  
0  cellular   10   nov       318         2     97         6  success  yes  
1  cellular    6   sep       245         1     -1         0  unknown  yes  
2  cellular   17   nov       369         1     -1         0  unknown   no  
3  cellular   17   jul        18         8     -1         0  unknown   no  
4  cellular   28   jul       496         2    182        11  success  yes  
Número de registros: 9000
Total de variables: 17
Tipo de cada variable:
age           int64
j

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Variable: balance
Asimetría original: 0.801429198170957
Asimetría después de la transformación: -0.26133929150357105
Variable: age
Asimetría original: 0.18159476272054004
Asimetría después de la transformación: -0.009233341952564379
Variable: day
Asimetría original: -0.1735647809482049
Asimetría después de la transformación: -0.17787175413317574
Variable: duration
Asimetría original: 0.13028827722652142
Asimetría después de la transformación: 0.009563647062496529
Variable: campaign
Asimetría original: 7.9814239758795145
Asimetría después de la transformación: 0.20064923802604986
Variable: pdays
Asimetría original: 7.28003559845937
Asimetría después de la transformación: 1.182197579917036
Variable: previous
Asimetría original: 0.028452063862845035
Asimetría después de la transformación: 1.1944941366807837
Variable 'balance' después de la transformación no tiene sesgo (asimetría: -0.26133929150357105)
Variable 'age' después de la transformación no tiene sesgo (asimetría: -0.0092333419525

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


Error Cuadrático Medio (MSE): 0.10777891231755032
Coeficiente de Determinación (R²): 0.555174952602348
Precisión del Modelo: 0.85
Matriz de Confusión:
[[908 150]
 [120 622]]
Columnas con valores NaN: ['balance_log']
Cantidad de valores NaN por columna:
 age                  0
job                  0
marital              0
education            0
default              0
balance              0
housing              0
loan                 0
contact              0
day                  0
month                0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
y                    0
balance_log        636
age_log              0
day_log              0
duration_log         0
campaign_log         0
pdays_log            0
previous_log         0
balance_boxcox       0
age_boxcox           0
day_boxcox           0
duration_boxcox      0
campaign_boxcox      0
pdays_boxcox         0
previous_boxcox      0
dtype: int64
   age  job  marital

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


Métricas de evaluación en el conjunto de validación:
Error Cuadrático Medio (MSE): 0.10777891231755032
Coeficiente de Determinación (R²): 0.555174952602348
Precisión del Modelo en el conjunto de validación: 0.85
Matriz de Confusión en el conjunto de validación:
[[908 150]
 [120 622]]

Métricas de evaluación en el conjunto de prueba:
Precisión del Modelo en el conjunto de prueba: 0.845
Matriz de Confusión en el conjunto de prueba:
[[850 155]
 [124 671]]
